# Yeast cells detection sample notebook

In [1]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.visualize as visualize

In [2]:
cd ~

/home/p301081


### Load data

In [3]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Movie_1/tiffs'
filenames = data.load_data(path)
print(filenames[0])

../../slow-data/yeastcells/data/DATA_CIT/Movie_1/tiffs/12.4.2017_sfp1_ph-tdgfp_hta2_mcherry_1pc_glu_to_1pc_glu_rap_xy01.nd2.tif


In [4]:
fn = filenames[0]
image = data.read_image(fn)
np.shape(image) #(frames, length, width, channels)

(217, 512, 512, 3)

### Load and run model on data for segmentation

In [5]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [6]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/detectron2/layers/wrappers.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  return x.nonzero().unbind(1)


### Run clustering algorithm for tracking

In [7]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.4, progress=False)

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/sklearn/neighbors/_base.py:168: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  EfficiencyWarning)


In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Extract features and visualize tracking and segmentation

In [8]:
s, t = features.get_seg_track(labels, output, frame = None) # total

The number of segmentations is 9852
The number of tracked cells is 357


In [10]:
s, t = features.get_seg_track(lables, output, frame = 1) # by frame

The number of segmentations is 98
The number of tracked cells is 52


In [ ]:
features.track_len(labels, label_num=0) #select a label to extract the length of its track

In [11]:
contours = features.extract_contours(output) 

In [ ]:
%matplotlib notebook
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=False)
movie = visualize.show_animation(scene, title='', delay=500) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(scene, coordinates, labels, w=40, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=500)

In [50]:
polygons_inst =  features.polygons_per_instance(contours, output)

In [ ]:
ax = visualize.plot_area_profile(polygons, label=5, ax=None, title=None)

In [51]:
_, _, pred_df = data.get_pred(output, labels, coordinates)
pred_features = features.get_features_df(polygons_inst, labels, pred_df)